# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')
%matplotlib inline

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [3]:
#Your code here
df = pd.read_csv('homepage_actions.csv')
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view
6,2016-09-24 20:39:03.248853,522116,experiment,view
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
9,2016-09-24 21:00:12.278374,560027,control,view


In [5]:
df.groupby('id').agg('|'.join).group.value_counts()
# Sean's method

control                  2400
experiment               2068
control|control           932
experiment|experiment     928
Name: group, dtype: int64

In [6]:
exp = df[df.group == 'experiment']
ctrl = df[df.group == 'control']

In [7]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [16]:
c_clicks = ctrl.action.value_counts().click
e_clicks = exp.action.value_counts().click

control_click_rate = c_clicks / ctrl.loc[ctrl['action']== 'view'].shape[0]
experiment_click_rate = e_clicks / exp.loc[exp['action']== 'view'].shape[0]

In [17]:
print(control_click_rate)

0.2797118847539016


In [18]:
print(experiment_click_rate)

0.3097463284379172


In [19]:
import scipy
import statsmodels
import statsmodels.stats
import statsmodels.stats.proportion
n0 = 2400 + 932 #control
n1 = 2068 + 928 #experimental
k0 = 932
k1 = 928
zstat, pvalue = statsmodels.stats.proportion.proportions_ztest([k0,k1],[n0,n1],value=0)
print('zstat',zstat)
print('pvalue',pvalue)

# ^Steps 1 - 3 

zstat -2.618563885349469
pvalue 0.008830075576595804


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [24]:
#Your code here
expected_clicks = exp.loc[exp['action']== 'view'].shape[0] * control_click_rate
# number of observations for the experiment * proportion of the control group

actual_clicks = exp.loc[exp['action']== 'view'].shape[0] * experiment_click_rate
# number of observations for the experiment * proportion of the experimental click rate

In [28]:
print(expected_clicks)

838.0168067226891


In [29]:
print(actual_clicks)

928.0


In [ ]:
# for bernoulli distributions expect the mu = proportion 

In [ ]:
# for binomial distributions expect mu = n * proportion

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [27]:
#Your code here
n = exp.loc[exp['action']== 'view'].shape[0]
p = control_click_rate

product = n*p*(1-p)
std_dev = np.sqrt(product)
std_dev

24.568547907005815

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [30]:
#Your code here
z_score = (actual_clicks - expected_clicks) / std_dev
z_score

3.6625360854823588

In [33]:
# When would we ever reduce the alpha from 0.05
# Answer = Whenever you want to reduce the risk of type I errors
# When would we ever increase the alpha above 0.05
# Answer = Whenever you want to reduce the risk of type II errors

In [32]:
p_value = stats.norm.sf(z_score)
p_value
# P-Value is much smaller than alpha, so we reject the null hypothesis

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.